In [1]:
import sys

if "google.colab" in sys.modules:
    !git clone https://github.com/rapidsai/rapidsai-csp-utils.git
    !python rapidsai-csp-utils/colab/pip-install.py
    !pip install faiss-cpu

Cloning into 'rapidsai-csp-utils'...
remote: Enumerating objects: 597, done.
remote: Counting objects: 100% (163/163), done.
remote: Compressing objects: 100% (81/81), done.
remote: Total 597 (delta 128), reused 82 (delta 82), pack-reused 434 (from 3)
Receiving objects: 100% (597/597), 196.59 KiB | 11.56 MiB/s, done.
Resolving deltas: 100% (302/302), done.
Installing RAPIDS remaining 25.04 libraries
Using Python 3.11.13 environment at: /usr
Resolved 175 packages in 9.39s
 Downloaded shapely
 Downloaded dask
 Downloaded datashader
 Downloaded bokeh
 Downloaded ucx-py-cu12
 Downloaded cudf-cu12
 Downloaded rmm-cu12
 Downloaded cuproj-cu12
 Downloaded pylibcugraph-cu12
 Downloaded libkvikio-cu12
 Downloaded cucim-cu12
 Downloaded pylibcudf-cu12
 Downloaded cuml-cu12
 Downloaded librmm-cu12
 Downloaded libraft-cu12
 Downloaded nvidia-nvcomp-cu12
 Downloaded cugraph-cu12
 Downloaded cuspatial-cu12
 Downloaded libcuspatial-cu12
 Downloaded raft-dask-cu12
 Downloaded libcuml-cu12
 Downloaded 

In [2]:
import cuvs
import faiss

cuvs_version = cuvs.__version__
faiss_cpu_version = faiss.__version__

print(f"cuVS version: {cuvs_version}")
print(f"FAISS CPU version: {faiss_cpu_version}")

cuVS version: 25.04.00
FAISS CPU version: 1.12.0


# 데이터 생성

In [3]:
import numpy as np
import cupy as cp

# 1. NumPy 시드 고정
np.random.seed(42)

# 2. 데이터 생성 : 분포가 다른 2개의 2차원 데이터를 합쳐서 벡터 크기 차이가 나게 만듦
dataset_np = np.hstack([
    np.random.random((500000, 128)),   # [0, 1) 구간의 균일 분포
    np.random.randn(500000, 128) * 5    # 평균 0, 표준편차 5인 정규 분포
]).astype(np.float32)

# 3. Numpy 배열을 CuPy 배열로 변환
dataset_cp = cp.asarray(dataset_np)

# 4. 쿼리 개수 지정
n_queries = 1000
query_np = dataset_np[:n_queries]
query_cp = dataset_cp[:n_queries].copy()

# 5. 데이터셋과 쿼리 벡터의 일부 출력
print("데이터셋 일부:\n", dataset_np[:1], end='\n\n')  # 첫 2개 샘플만 출력
print("쿼리 벡터:\n", query_np)

데이터셋 일부:
 [[ 3.74540120e-01  9.50714290e-01  7.31993914e-01  5.98658502e-01
   1.56018645e-01  1.55994520e-01  5.80836125e-02  8.66176128e-01
   6.01114988e-01  7.08072603e-01  2.05844939e-02  9.69909847e-01
   8.32442641e-01  2.12339118e-01  1.81824967e-01  1.83404505e-01
   3.04242253e-01  5.24756432e-01  4.31945026e-01  2.91229129e-01
   6.11852884e-01  1.39493868e-01  2.92144656e-01  3.66361856e-01
   4.56069976e-01  7.85175979e-01  1.99673787e-01  5.14234424e-01
   5.92414558e-01  4.64504138e-02  6.07544839e-01  1.70524120e-01
   6.50515929e-02  9.48885560e-01  9.65632021e-01  8.08397353e-01
   3.04613769e-01  9.76721123e-02  6.84233010e-01  4.40152496e-01
   1.22038238e-01  4.95176911e-01  3.43885198e-02  9.09320414e-01
   2.58779973e-01  6.62522256e-01  3.11711073e-01  5.20068049e-01
   5.46710253e-01  1.84854463e-01  9.69584644e-01  7.75132835e-01
   9.39498961e-01  8.94827366e-01  5.97899973e-01  9.21874225e-01
   8.84925053e-02  1.95982859e-01  4.52272892e-02  3.25330317e-01


# FAISS HNSW-Flat

In [4]:
%%time

import numpy as np
import faiss

# 1. 코사인 유사도 계산을 위한 L2 정규화
faiss.normalize_L2(dataset_np)
faiss.normalize_L2(query_np)

# 2. 파라미터 설정
k = 10                               # Top-k
d = dataset_np.shape[1]              # 특성 차원(열 개수)
M = 64                               # 그래프에서 각 노드가 연결할 최대 이웃 수
ef_search = 256                      # 탐색 시 고려할 후보 개수
metric = faiss.METRIC_INNER_PRODUCT  # L2 정규화된 벡터 간 내적으로 코사인 유사도 계산

# 3. HNSW-Flat 인덱스 생성
index = faiss.IndexHNSWFlat(d, M, metric)

# 4. 인덱스에 벡터 추가
index.add(dataset_np)

# 5. 탐색 성능 파라미터 설정
index.hnsw.efSearch = ef_search

# 6. 벡터 검색
distances, neighbors = index.search(query_np, k)

# 7. 1000개의 쿼리의 검색 결과 출력
for i in range(len(query_np)):
    print(f"\n쿼리 {i}:")
    print("Top-k 인덱스:", neighbors[i])
    print("유사도 점수 (cosine):", distances[i])

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
Top-k 인덱스: [     0 265723 455839 393268 163835 101190 434104  63911 258663 388127]
유사도 점수 (cosine): [1.0000001  0.38176894 0.3625757  0.3612368  0.35987377 0.35840312
 0.3521975  0.34877378 0.3483927  0.34697157]

쿼리 1:
Top-k 인덱스: [     1 460724  25655 303227  11058 392494 391519 131027 427829  67350]
유사도 점수 (cosine): [1.0000001  0.44864494 0.39349487 0.38874245 0.3744548  0.37141228
 0.3545121  0.35239    0.35190263 0.35154638]

쿼리 2:
Top-k 인덱스: [     2 494638 470370 368153 335772 233520 383608 472101 329372 159113]
유사도 점수 (cosine): [1.         0.39540836 0.37565956 0.3613519  0.3504479  0.3428613
 0.33546942 0.33346027 0.33259362 0.3315388 ]

쿼리 3:
Top-k 인덱스: [     3 178740  26058 213422 431259 312903 348615 230842 367027  66133]
유사도 점수 (cosine): [1.0000001  0.37751222 0.3744778  0.3717479  0.36836845 0.36128289
 0.3572359  0.35584092 0.34715977 0.34669554]

쿼리 4:
Top-k 인덱스: [     4 288098 393223 428493  35574 261073 465614 344264 494268 313818]
유사

# cuVS CAGRA

In [5]:
def l2_normalize_cp(x, axis=1, eps=1e-12):
    norm = cp.linalg.norm(x, axis=axis, keepdims=True)
    return x / (norm + eps)

In [6]:
%%time

import cupy as cp
from cuvs.neighbors import cagra

# 1. 파라미터 설정
k = 10                               # Top-k
M = 64                               # 그래프에서 각 노드가 연결할 최대 이웃 수
ef_search = 256                      # 탐색 시 고려할 후보 개수

# 2. 코사인 유사도 계산을 위한 L2 정규화
normalized_dataset_cp = l2_normalize_cp(dataset_cp)
normalized_query_cp = l2_normalize_cp(query_cp)

# 3. CAGRA 인덱스 생성 파라미터 설정 (L2 거리 기반)
build_params = cagra.IndexParams(
    metric="inner_product",
    graph_degree=M
    )

# 4. 인덱스 빌드
index = cagra.build(build_params, normalized_dataset_cp)

# 5. 벡터 검색
search_params = cagra.SearchParams(itopk_size=ef_search)
distances, neighbors = cagra.search(search_params, index, normalized_query_cp, k)

# 6. 검색 결과를 CuPy 배열로 변환 (선택적)
distances = cp.asarray(distances)
neighbors = cp.asarray(neighbors)

# 7. 1000개의 쿼리의 검색 결과 출력
for i in range(len(normalized_query_cp)):
    print(f"\n쿼리 {i}:")
    print("Top-k 인덱스:", neighbors[i])
    print("유사도 점수 (cosine):", distances[i])

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
Top-k 인덱스: [     0 499700 129395  76297 455839 469980 434104 258663 106117 378646]
유사도 점수 (cosine): [0.9999999  0.4143617  0.37672567 0.37299109 0.36257565 0.35666043
 0.3521974  0.34839267 0.34559825 0.34548628]

쿼리 1:
Top-k 인덱스: [     1 460724  24702  25655 119095 327550 178862 400222 403147 423705]
유사도 점수 (cosine): [1.         0.44864494 0.40347752 0.39349484 0.3698097  0.3692933
 0.36499286 0.35941368 0.35868543 0.358553  ]

쿼리 2:
Top-k 인덱스: [     2 494638 105816 223728 283080 235398  20733 489187 430791 369591]
유사도 점수 (cosine): [1.         0.3954084  0.38268462 0.37833053 0.3693082  0.36460614
 0.36019927 0.3593589  0.35871243 0.35301393]

쿼리 3:
Top-k 인덱스: [     3  26058 251784 213422 431259 112555  91802 312903  33377 367027]
유사도 점수 (cosine): [1.         0.3744778  0.37387013 0.37174788 0.36836845 0.3656197
 0.3651097  0.36128283 0.35115236 0.34715974]

쿼리 4:
Top-k 인덱스: [     4 288098 393223 122011 242418 398628 381460 428493 248351  23142]
유사도